In [2]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
from tensorflow.keras.utils import Progbar

import transformers
from transformers import TFBertForPreTraining
from transformers import TFTrainer
from transformers import BertConfig, BertTokenizer

In [5]:
AVAILABLE =  {'af','ar','bg','bn','br','bs','ca','cs',
              'da','de','el','en','eo','es','et','eu',
              'fa','fi','fr','gl','he','hi','hr','hu',
              'hy','id','is','it','ja','ka','kk','ko',
              'lt','lv','mk','ml','ms','nl','no','pl',
              'pt','pt_br','ro','ru','si','sk','sl','sq',
              'sr','sv','ta','te','th','tl','tr','uk',
              'ur','vi','ze_en','ze_zh','zh','zh_cn',
              'zh_en','zh_tw','zh_zh'}

LANG_CODE = "en" #@param {type:"string"}

assert LANG_CODE in AVAILABLE, "Invalid language code selected"

!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2016/mono/OpenSubtitles.raw.'$LANG_CODE'.gz -O dataset.txt.gz
!gzip -d dataset.txt.gz
!tail dataset.txt

zsh:1: no matches found: http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2016/mono/OpenSubtitles.raw.$LANG_CODE.gz
gzip: can't stat: dataset.txt.gz (dataset.txt.gz.gz): No such file or directory
tail: dataset.txt: No such file or directory


In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForPreTraining.from_pretrained('bert-base-uncased')
input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute",add_special_tokens=True))[None,:]
print(input_ids)
outputs = model(input_ids)
prediction_scores, seq_relationship_scores = outputs[:2]
print()
print(prediction_scores)
print()
print(seq_relationship_scores)

All model checkpoint layers were used when initializing TFBertForPreTraining.

All the layers of TFBertForPreTraining were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


tf.Tensor([[  101  7592  1010  2026  3899  2003 10140   102]], shape=(1, 8), dtype=int32)

tf.Tensor(
[[[ -7.8961954  -7.8105164  -7.7903347 ...  -7.069408   -7.1693287
    -4.3589554]
  [ -8.446108   -8.440074   -8.504436  ...  -8.062478   -7.990895
    -5.715992 ]
  [-15.295272  -15.472722  -15.586455  ... -12.985673  -11.703852
   -11.429297 ]
  ...
  [-14.062792  -14.253545  -14.364464  ... -12.715123  -11.1620865
   -10.231738 ]
  [-10.657623  -10.789212  -11.0402155 ... -10.323338  -10.157843
    -3.7721484]
  [-11.338303  -11.459005  -11.17674   ...  -9.215169   -9.520874
    -9.557125 ]]], shape=(1, 8, 30522), dtype=float32)

tf.Tensor([[ 3.3474083 -2.0612679]], shape=(1, 2), dtype=float32)


In [9]:
import nltk.corpus
from nltk.corpus import brown
born.words()

ModuleNotFoundError: No module named 'nltk.corpus'

In [1]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size= 52000,
    max_position_embeddings= 514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [3]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base',max_len=512)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [5]:
model.num_parameters()

83504416

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset